In [1]:
# repo rootz
%cd /work

/work


In [2]:
!ls 

LICENSE    docker    out    pyproject.toml  scripts  tests
README.md  examples  paper  resources	    src


In [3]:
#!chmod +x resources/gene_id_maps/build_entrez_id_map.sh
#! ./resources/gene_id_maps/build_entrez_id_map.sh

In [4]:
#!chmod +x resources/gene_id_maps/build_ensembl_to_symbol_from_gtf.sh
#! ./resources/gene_id_maps/build_ensembl_to_symbol_from_gtf.sh --download

In [5]:
# Fetch from UCSC Xena
!python paper/scripts/fig2_fetch_inputs.py

[fetch_inputs] OK
  /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz
  /work/paper/source_data/PANCAN_TP53_v1/raw/mc3.v0.2.8.PUBLIC.xena.gz
  /work/paper/source_data/PANCAN_TP53_v1/raw/TCGA_phenotype_dense.tsv.gz


In [5]:
# Extract TP53 mutations (mc3.v0.2.8.PUBLIC.xena.gz)
# Create barcode→cancer from phenotype 
# Output derived/groups/{CANCER}.groups.tsv　for each cancer type

!python paper/scripts/fig2_make_groups.py

[make_groups] sample_type_id raw unique (head): ['07', '06', '05', '03', '02', '01', '11', '']
[make_groups] sample_type_id norm counts: {'1': 10593, '11': 1475, '6': 396, '3': 200, '': 72, '2': 56, '5': 11, '7': 1}
[make_groups] filtered to primary-like sample_type_id in {1,3}: 10793
[make_groups] phenotype mapped samples: 10793
[make_groups] cancers: ['ACC', 'BLCA', 'BRCA', 'CESC', 'CHOL', 'COAD', 'DLBC', 'ESCA', 'GBM', 'HNSC', 'KICH', 'KIRC', 'KIRP', 'LAML', 'LGG', 'LIHC', 'LUAD', 'LUSC', 'MESO', 'OV', 'PAAD', 'PCPG', 'PRAD', 'READ', 'SARC', 'SKCM', 'STAD', 'TGCT', 'THCA', 'THYM', 'UCEC', 'UCS', 'UVM']
[make_groups] MC3 TP53 rows: 3729
[make_groups] MC3 TP53 protein-altering rows (pre-FILTER): 3620
[make_groups] FILTER column not found; not applying FILTER
[make_groups] WARNING: LAML has n_mut=0, n_wt=200
[make_groups] WARNING: UVM has n_mut=0, n_wt=80
[make_groups] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/derived/groups/PANCAN.groups.tsv
  cancers: 33
  samples (mapped): 

In [3]:
!rm -rf /work/paper/source_data/PANCAN_TP53_v1/sample_cards/*
!python paper/scripts/fig2_make_sample_cards.py

[make_sample_cards] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/sample_cards (33 cancers × 2 cards)


In [4]:
!cat /work/paper/source_data/PANCAN_TP53_v1/sample_cards/HNSC.note.sample_card.json

{
  "comparison": "TP53_mut_vs_TP53_wt",
  "condition": "HNSC",
  "extra": {
    "audit_tau": 0.8,
    "benchmark_id": "PANCAN_TP53_v1",
    "context_gate_mode": "note",
    "gene_id_map_tsv": "resources/gene_id_maps/id_map.tsv.gz",
    "goal": "context-conditioned pathway claim selection with calibrated abstention"
  },
  "k_claims": 100,
  "notes": "TCGA Pan-Cancer; groups defined from MC3 TP53 PASS mutations.",
  "perturbation": "genotype",
  "tissue": "tumor"
}


In [5]:
!cat /work/paper/source_data/PANCAN_TP53_v1/sample_cards/HNSC.hard.sample_card.json

{
  "comparison": "TP53_mut_vs_TP53_wt",
  "condition": "HNSC",
  "extra": {
    "audit_tau": 0.8,
    "benchmark_id": "PANCAN_TP53_v1",
    "context_gate_mode": "hard",
    "gene_id_map_tsv": "resources/gene_id_maps/id_map.tsv.gz",
    "goal": "context-conditioned pathway claim selection with calibrated abstention"
  },
  "k_claims": 100,
  "notes": "TCGA Pan-Cancer; groups defined from MC3 TP53 PASS mutations.",
  "perturbation": "genotype",
  "tissue": "tumor"
}


In [10]:
for c in ("HNSC", "LUAD", "LUSC", "BRCA", "OV",  "UCEC",  "SKCM"):
  !Rscript paper/scripts/fig2_deg_rank.R $c


[deg_rank] inputs
  cancer: HNSC 
  expr: /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz 
  groups: /work/paper/source_data/PANCAN_TP53_v1/derived/groups/HNSC.groups.tsv 
[deg_rank] matched samples: 520  / expr= 11069  / groups= 528 
[deg_rank] QC: dropping zero-variance genes: 269 
[deg_rank] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/HNSC.deg_ranking.tsv 
  n_genes: 20262 
[deg_rank] inputs
  cancer: LUAD 
  expr: /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz 
  groups: /work/paper/source_data/PANCAN_TP53_v1/derived/groups/LUAD.groups.tsv 
[deg_rank] matched samples: 515  / expr= 11069  / groups= 520 
[deg_rank] QC: dropping zero-variance genes: 338 
[deg_rank] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/LUAD.deg_ranking.tsv 
  n_genes: 20193 
[deg_rank] inputs
  cancer: LUSC 
  expr: /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz 
  groups: /work/paper/source_data/PANCAN_TP53_v1/derived/gr

In [6]:
# Hallmark
for c in ("HNSC", "LUAD", "LUSC", "BRCA", "OV",  "UCEC",  "SKCM"):
  !Rscript paper/scripts/figSx_fgsea_to_evidence_table.R $c --collection H


[fgsea_to_evidence] inputs
  cancer: HNSC 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/HNSC.deg_ranking.tsv 
  out : /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.H.evidence_table.tsv 
  collection: H 
  subcategory: <none> 
  source_tag: fgsea_msigdb_H_entrez 
  msigdbr_version: 25.1.1 
  fgsea_version  : 1.36.2 
Warning message:
The `category` argument of `msigdbr()` is deprecated as of msigdbr 10.0.0.
ℹ Please use the `collection` argument instead. 
[fgsea_to_evidence] msigdbr entrez column: entrez_gene 
[fgsea_to_evidence] overlap diagnostics
  overlap_summary: min 24 p25 80 median 141 p75 165 max 193 
  pathways_with_overlap>=15: 50 / 50 
[fgsea_to_evidence] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.H.evidence_table.tsv 
  n_terms: 50 
[fgsea_to_evidence] inputs
  cancer: LUAD 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/LUAD.deg_ranking.tsv 
  out : /work/paper/source_data/PANCAN_TP53_v1/evidence_

In [7]:
# GO
for c in ("HNSC", "LUAD", "LUSC", "BRCA", "OV",  "UCEC",  "SKCM"):
   !Rscript paper/scripts/figSx_fgsea_to_evidence_table.R $c --collection C5 --subcategory GO:BP


[fgsea_to_evidence] inputs
  cancer: HNSC 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/HNSC.deg_ranking.tsv 
  out : /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.C5_GO_BP.evidence_table.tsv 
  collection: C5 
  subcategory: GO:BP 
  source_tag: fgsea_msigdb_C5_GO_BP_entrez 
  msigdbr_version: 25.1.1 
  fgsea_version  : 1.36.2 
Warning message:
The `category` argument of `msigdbr()` is deprecated as of msigdbr 10.0.0.
ℹ Please use the `collection` argument instead. 
[fgsea_to_evidence] msigdbr entrez column: entrez_gene 
[fgsea_to_evidence] overlap diagnostics
  overlap_summary: min 0 p25 6 median 13 p75 38 max 1258 
  pathways_with_overlap>=15: 3525 / 7583 
[fgsea_to_evidence] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.C5_GO_BP.evidence_table.tsv 
  n_terms: 3386 
[fgsea_to_evidence] inputs
  cancer: LUAD 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/LUAD.deg_ranking.tsv 
  out : /work/paper/source_data/

In [12]:
# Reactome / KEGG
for c in ("HNSC", "LUAD", "LUSC", "BRCA", "OV",  "UCEC",  "SKCM"):
  !Rscript paper/scripts/figSx_fgsea_to_evidence_table.R $c --collection C2 --subcategory CP:REACTOME
  !Rscript paper/scripts/figSx_fgsea_to_evidence_table.R $c --collection C2 --subcategory CP:KEGG_MEDICUS



[fgsea_to_evidence] inputs
  cancer: HNSC 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/HNSC.deg_ranking.tsv 
  out : /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.C2_CP_REACTOME.evidence_table.tsv 
  collection: C2 
  subcategory: CP:REACTOME 
  source_tag: fgsea_msigdb_C2_CP_REACTOME_entrez 
  msigdbr_version: 25.1.1 
  fgsea_version  : 1.36.2 
Warning message:
The `category` argument of `msigdbr()` is deprecated as of msigdbr 10.0.0.
ℹ Please use the `collection` argument instead. 
[fgsea_to_evidence] msigdbr entrez column: entrez_gene 
[fgsea_to_evidence] overlap diagnostics
  overlap_summary: min 0 p25 9 median 19 p75 45 max 922 
  pathways_with_overlap>=15: 1036 / 1787 
[fgsea_to_evidence] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.C2_CP_REACTOME.evidence_table.tsv 
  n_terms: 984 
[fgsea_to_evidence] inputs
  cancer: HNSC 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/HNSC.deg_ranking.tsv 
  out : /w

In [225]:
# Fig2 run
  #--cancers HNSC,LUAD,LUSC,BRCA,OV,UCEC,SKCM \
!python paper/scripts/run_figSx_collections_pipeline.py \
  --benchmark-id PANCAN_TP53_v1 \
  --cancers HNSC,LUAD,LUSC,BRCA,OV,UCEC,SKCM \
  --collections Hallmark,GO_BP,Reactome,KEGG_MEDICUS \
  --variants ours \
  --gate-modes hard \
  --tau 0.2 \
  --k-claims 50 \
  --out-root "/work/paper/source_data/PANCAN_TP53_v1/out_supp_collections_tau_0.20" \
  --force


[run_figSx] plan
  benchmark_id: PANCAN_TP53_v1
  tau: 0.20
  cancers: HNSC,LUAD,LUSC,BRCA,OV,UCEC,SKCM
  collections: Hallmark,GO_BP,Reactome,KEGG_MEDICUS
  variants: ours
  gate_modes: hard
  k_claims: 50
  seed: 42
  out_root: /work/paper/source_data/PANCAN_TP53_v1/out_supp_collections_tau_0.20
[run_figSx] jobs: 28
[run_figSx] (1/28) Hallmark HNSC ours gate=hard tau=0.20
[run_figSx] (2/28) Hallmark LUAD ours gate=hard tau=0.20
[run_figSx] (3/28) Hallmark LUSC ours gate=hard tau=0.20
[run_figSx] (4/28) Hallmark BRCA ours gate=hard tau=0.20
[run_figSx] (5/28) Hallmark OV ours gate=hard tau=0.20
[run_figSx] (6/28) Hallmark UCEC ours gate=hard tau=0.20
[run_figSx] (7/28) Hallmark SKCM ours gate=hard tau=0.20
[run_figSx] (8/28) GO_BP HNSC ours gate=hard tau=0.20
[run_figSx] (9/28) GO_BP LUAD ours gate=hard tau=0.20
[run_figSx] (10/28) GO_BP LUSC ours gate=hard tau=0.20
[run_figSx] (11/28) GO_BP BRCA ours gate=hard tau=0.20
[run_figSx] (12/28) GO_BP OV ours gate=hard tau=0.20
[run_figSx] 

In [226]:
!python paper/scripts/figSx_collect_collection_metrics.py \
  --out-root /work/paper/source_data/PANCAN_TP53_v1/out_supp_collections_tau_0.20 \
  --benchmark-id PANCAN_TP53_v1 \
  --conditions HNSC,LUAD,LUSC,BRCA,OV,UCEC,SKCM \
  --variants ours \
  --gate-modes hard \
  --taus 0.2


WROTE:
   /work/paper/source_data/PANCAN_TP53_v1/collection_metrics/collections_summary_wide.tsv
   /work/paper/source_data/PANCAN_TP53_v1/collection_metrics/module_survival_long.tsv
   /work/paper/source_data/PANCAN_TP53_v1/collection_metrics/audit_reasons_long.tsv


In [227]:
!ls  /work/paper/source_data/BEATAML_TP53_v1/out/BEATAML/ours/gate_hard/tau_0.82

ls: cannot access '/work/paper/source_data/BEATAML_TP53_v1/out/BEATAML/ours/gate_hard/tau_0.82': No such file or directory


In [228]:
!python paper/scripts/figSx_plot_collection_panels.py \
  --collections-summary-wide /work/paper/source_data/PANCAN_TP53_v1/collection_metrics/collections_summary_wide.tsv \
  --module-survival-long     /work/paper/source_data/PANCAN_TP53_v1/collection_metrics/module_survival_long.tsv \
  --audit-reasons-long       /work/paper/source_data/PANCAN_TP53_v1/collection_metrics/audit_reasons_long.tsv \
  --outdir /work/paper/source_data/PANCAN_TP53_v1/fig/SuppFigSx_tau0.20 \
  --fmt pdf \
  --conditions HNSC,LUAD,LUSC,BRCA,OV,UCEC,SKCM \
  --variants ours \
  --gate-modes hard \
  --taus 0.2 \
  --include-fail-reasons


[figSx_plot] WROTE:
  /work/paper/source_data/PANCAN_TP53_v1/fig/SuppFigSx_tau0.20/FigSx_PanelA_module_survival.pdf
  /work/paper/source_data/PANCAN_TP53_v1/fig/SuppFigSx_tau0.20/FigSx_PanelB_module_stats.pdf
  /work/paper/source_data/PANCAN_TP53_v1/fig/SuppFigSx_tau0.20/FigSx_PanelC_outcomes.pdf
  /work/paper/source_data/PANCAN_TP53_v1/fig/SuppFigSx_tau0.20/FigSx_PanelD_top_reasons.pdf


In [109]:
!ls /work/paper/source_data/PANCAN_TP53_v1/out_supp_collections_tau_0.20/

C2_CP_KEGG_MEDICUS  C2_CP_REACTOME  C5_GO_BP  H


In [108]:
!python3 paper/scripts/figSx_collections_plot_hallmark_threshold.py \
  --distilled /work/paper/source_data/PANCAN_TP53_v1/out_supp_collections_tau_0.20/H/HNSC/ours/gate_hard/tau_0.20/distilled.tsv \
  --outdir /work/paper/source_data/PANCAN_TP53_v1/fig/SuppFigSx_tau0.20 \
  --fmt pdf \
  --max-gene-term-degree 200 \
  --min-shared-genes 3 \
  --jaccard-min 0.10 \
  --hub-degree-for-filter 200 \
  --pair-sample-max 200000 \
  --seed 0


[figSx_hallmark_thresholds] WROTE:
  /work/paper/source_data/PANCAN_TP53_v1/fig/SuppFigSx_tau0.20/FigSx_HallmarkThresholds_PanelA_gene_degree.pdf
  /work/paper/source_data/PANCAN_TP53_v1/fig/SuppFigSx_tau0.20/FigSx_HallmarkThresholds_PanelB_shared_genes.pdf
  /work/paper/source_data/PANCAN_TP53_v1/fig/SuppFigSx_tau0.20/FigSx_HallmarkThresholds_PanelC_jaccard.pdf
  /work/paper/source_data/PANCAN_TP53_v1/fig/SuppFigSx_tau0.20/FigSx_hallmark_threshold_percentiles.tsv
  /work/paper/source_data/PANCAN_TP53_v1/fig/SuppFigSx_tau0.20/FigSx_hallmark_threshold_percentiles.json


In [145]:
!python3 paper/scripts/figSx_collections_plot_hallmark_threshold.py \
  --distilled /work/paper/source_data/PANCAN_TP53_v1/out_supp_collections_tau_0.20/C2_CP_KEGG_MEDICUS/HNSC/ours/gate_hard/tau_0.20/distilled.tsv \
  --outdir /work/paper/source_data/PANCAN_TP53_v1/fig/SuppFigSx_tau0.20 \
  --fmt pdf \
  --max-gene-term-degree 200 \
  --min-shared-genes 3 \
  --jaccard-min 0.10 \
  --hub-degree-for-filter 200 \
  --pair-sample-max 200000 \
  --seed 0


[figSx_hallmark_thresholds] WROTE:
  /work/paper/source_data/PANCAN_TP53_v1/fig/SuppFigSx_tau0.20/FigSx_HallmarkThresholds_PanelA_gene_degree.pdf
  /work/paper/source_data/PANCAN_TP53_v1/fig/SuppFigSx_tau0.20/FigSx_HallmarkThresholds_PanelB_shared_genes.pdf
  /work/paper/source_data/PANCAN_TP53_v1/fig/SuppFigSx_tau0.20/FigSx_HallmarkThresholds_PanelC_jaccard.pdf
  /work/paper/source_data/PANCAN_TP53_v1/fig/SuppFigSx_tau0.20/FigSx_hallmark_threshold_percentiles.tsv
  /work/paper/source_data/PANCAN_TP53_v1/fig/SuppFigSx_tau0.20/FigSx_hallmark_threshold_percentiles.json
